In [ ]:
!pip install -qU langchain_upstage
!pip install -qU kiwipiepy
!pip install -qU langchain_chroma
!pip install -qU langchain_community
!pip install -qU rank_bm25

In [2]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.tokenize("안녕하세요? 형태소 분석기 키위입니다.")

[Token(form='안녕', tag='NNG', start=0, len=2),
 Token(form='하', tag='XSA', start=2, len=1),
 Token(form='세요', tag='EF', start=3, len=2),
 Token(form='?', tag='SF', start=5, len=1),
 Token(form='형태소', tag='NNG', start=7, len=3),
 Token(form='분석기', tag='NNG', start=11, len=3),
 Token(form='키위', tag='NNG', start=15, len=2),
 Token(form='이', tag='VCP', start=17, len=1),
 Token(form='ᆸ니다', tag='EF', start=17, len=3),
 Token(form='.', tag='SF', start=20, len=1)]

In [3]:
from langchain_core.documents import Document

docs = [
    Document(
        page_content="금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다."
    ),
    Document(
        page_content="금융저축보험은 규칙적인 저축을 통해 목돈을 마련할 수 있으며, 생명보험 기능도 겸비하고 있습니다."
    ),
    Document(
        page_content="저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다."
    ),
    Document(
        page_content="금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다."
    ),
    Document(
        page_content="금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다."
    ),
    Document(
        page_content="금보험은 저축성과를 극대화합니다. 특히 노후 대비 저축에 유리하게 구성되어 있습니다."
    ),
    Document(
        page_content="금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요."
    ),
]

In [4]:
def kiwi_toknize(text):
    return [token.form for token in kiwi.tokenize(text)]

kiwi_toknize(docs[0].page_content)

['금융',
 '보험',
 '은',
 '장기',
 '적',
 '이',
 'ᆫ',
 '자산',
 '관리',
 '와',
 '위험',
 '대비',
 '를',
 '목적',
 '으로',
 '고안',
 '되',
 'ᆫ',
 '금융',
 '상품',
 '이',
 'ᆸ니다',
 '.']

In [5]:
from langchain_community.retrievers import BM25Retriever

bm25 = BM25Retriever.from_documents(docs)
kiwi_bm25 = BM25Retriever.from_documents(docs, preprocess_func=kiwi_toknize)

In [8]:
def print_search_results(query):
    print(f"Query: {query}")
    print(f"BM25: {bm25.invoke(query)[0].page_content}")
    print(f"Kiwi_BM25: {kiwi_bm25.invoke(query)[0].page_content}")
    print("")


In [9]:

print_search_results("금융보험")
print_search_results("금융 보험")
print_search_results("금융저축보험")
print_search_results("축산물 보험")
print_search_results("저축금융보험")
print_search_results("금융보씨 개인정보 조회")

Query: 금융보험
BM25: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
Kiwi_BM25: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.

Query: 금융 보험
BM25: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
Kiwi_BM25: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.

Query: 금융저축보험
BM25: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
Kiwi_BM25: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.

Query: 축산물 보험
BM25: 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.
Kiwi_BM25: 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.

Query: 저축금융보험
BM25: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
Kiwi_BM25: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.

Query: 금융보씨 개인정보 조회
BM25: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
Kiwi_BM25: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.

